In [6]:
    import IPython, sys, pyld, copy, fnmatch, types, mypy.main, importlib, sys, contextlib, collections, inspect, pydantic, ipywidgets, pathlib, importlib_resources, abc, itertools, urllib, importnb,contextlib, collections, requests_file, requests_cache, fnmatch, traitlets, mimetypes, operator, importlib, requests, requests_cache, urllib, ast, abc,  itertools, mimetypes, io, functools, typing
    shell = get_ipython()
    CTX, ANNO = '@context __annotations__'.split()
    __all__ = 'context', 'app'
    

    import retype, black, monkeytype.tracing, monkeytype.stubs

    retype.Config.incremental = True

    import mypy.main, importlib, sys, contextlib, collections

    with __import__('importnb').Notebook():
        try: from . import adapters
        except: import adapters

In [7]:
    def expand(str, ctx):
        object = pyld.jsonld.expand({str: "", "@context": copy.copy(ctx)})
        if object: return next(iter(object[0]))
        return str

In [8]:
    def treat_path(object):
        try:
            path = pathlib.Path(object)
            if path.exists(): return str(path.absolute())
        except: ...
        return object

In [9]:
    mimetypes = __import__('mimetypes').MimeTypes()
    try:
        with importlib_resources.path('annodize', 'base.types') as path: mimetypes.read(path)
    except:
        if __name__ == '__main__': mimetypes.read('base.types')

In [10]:
    def annodize_response(response):
        callable = getattr(response, 'callable', None)
        response.object = getattr(response, 'object', None)
        if object is not None: response.object = callable(response) if callable else response        
        if response.status_code == requests.codes.ok:                    
            if isinstance(response.object, type(response)):
                try: return IPython.display.JSON(response.json())
                except: ...
            else: return response.object
        else: return IPython.display.Markdown(response.url)

In [11]:
    class RequestFormatter(IPython.core.formatters.BaseFormatter, requests.Session):
        parent = traitlets.Any()
        mimetypes = traitlets.Instance(__import__('mimetypes').MimeTypes, default_value=mimetypes)
        _return_type = object

        @traitlets.default('mimetypes')
        def _default_mimetypes(Mime):  return mimetypes

        adapter_list = traitlets.List()
        
        @traitlets.default('adapter_list')
        def _default_adapters(RequestFormatter, *object):
            http = adapters.HttpCache('local_cache')
            for adapter in [adapters.PythonAdapter(), adapters.Noop(), adapters.FileAdapter(), http, adapters.ContentsAdapter()]:
                for protocol in adapter.protocol.split(): object += [protocol, adapter],
            return list(object)
                    
        type_printers = traitlets.Dict({requests.Response: annodize_response})
        def __init__(RequestFormatter, *args, **kwargs):
            IPython.core.formatters.BaseFormatter.__init__(RequestFormatter, **kwargs), requests.Session.__init__(RequestFormatter)
            for key, value in RequestFormatter.adapter_list: RequestFormatter.mount(key, value)
            RequestFormatter.hooks['response'].append(RequestFormatter.post_response)
            
        def post_response(RequestFormatter, response, *x, **kwargs):
            response.headers['Content-Type'] =  RequestFormatter.mimetypes.guess_type(response.url)[0] or response.headers.get('Content-Type', '').partition(';')[0]
            try: response.callable= RequestFormatter.wrapped_request_response(RequestFormatter.lookup(response.headers['Content-Type']))
            except KeyError: ...
            return response            
        
        def __call__(RequestFormatter, object, ctx=None, name=None):
            if isinstance(object, str):
                ctx = copy.copy(ctx or getattr(RequestFormatter.parent, ANNO, {}).get(CTX, {}))
                for key in list(ctx.keys()):
                    isinstance(key, (str, dict)) or ctx.pop('key')
                try:
                    object = RequestFormatter.get(expand(treat_path(object), ctx)) if isinstance(object, str) else object
                except (requests.exceptions.MissingSchema, ValueError): ...
            return super().__call__(object)
        
        def lookup(RequestFormatter, object):
            obj_id = id(object)
            if obj_id in RequestFormatter.singleton_printers: return RequestFormatter.singleton_printers[obj_id]
            if not isinstance(object, str): object = IPython.core.formatters._get_type(object)
            return RequestFormatter.lookup_by_type(object)
    
        def lookup_by_type(RequestFormatter, type):
            if isinstance(type, str) and (type, None) in RequestFormatter.deferred_printers: 
                return RequestFormatter.deferred_printers[(type, None)]
            return super().lookup_by_type(type)
            
        def for_type_by_ext(RequestFormatter, ext, func=None):
            return AnnotationFormatter.for_type_by_name(*for_key_by_ext(ext, RequestFormatter.mimetypes), func)

        @staticmethod
        def wrapped_request_response(callable):
            @functools.wraps(callable)
            def call(response):
                if len(inspect.signature(callable).parameters) == 1: return callable(response.request)
                return callable(response.request, response)
            return call

In [14]:
    def alt(req, resp): return __import__('altair').Chart(**resp.json())

    def sqlite(req, resp):
        parsed = urllib.parse.urlparse(response.url)
        # do something for remotes here
        scheme, sep, path = resp.url.partition('://')
        connection = __import__('ibis').sqlite.connect(path)
        return F"{response.url} has tables named: {', '.join(connection.list_tables())}"
    
    def read_pandas(attr, request, response): 
        return getattr(__import__('pandas'), F"read_{attr}")(io.BytesIO(response.content)).sample(4)


    def iframe(req): return IPython.display.IFrame(req.url, '100%', '600')

    def for_key_by_ext(ext, types=None): return (types or mimetypes).guess_type(F"foo.{ext}")

    default_printers = RequestFormatter.deferred_printers.default_args[0]

    default_printers[(None, None)] = default_printers[for_key_by_ext('com')] = iframe
    default_printers[for_key_by_ext('py')] = lambda req, resp: IPython.display.Code(resp.text, language='python')
    default_printers[for_key_by_ext('md')] = lambda req, resp: IPython.display.Markdown(resp.text)
    for key in "csv feather parquet".split():
        default_printers[for_key_by_ext(key)] = functools.partial(read_pandas, key)
    default_printers[for_key_by_ext('json')] = lambda req, resp: IPython.display.JSON(resp.json())
    default_printers[for_key_by_ext('ipynb')] = lambda req, resp: IPython.display.JSON(resp.json())
    default_printers[for_key_by_ext('yaml')] = lambda req, resp: IPython.display.JSON(Yaml%resp.text)
    default_printers[for_key_by_ext('geojson')] = lambda req, resp: IPython.display.GeoJSON(resp.json())

    default_printers[for_key_by_ext('html')] = lambda req, resp: IPython.display.HTML(F"""<iframe srcdoc="{__import__('html').escape(resp.text)}" width="100%" height="600"/>""")
    for key in "jpg png bmp".split():
        default_printers[for_key_by_ext(key)] = lambda req, resp: IPython.display.Image(data=resp.content)
    default_printers[for_key_by_ext('vl')] = alt
    default_printers[for_key_by_ext('sqlite')] = sqlite
    default_printers[('', None)] = lambda request: IPython.display.Markdown(F"* {request.url}")

In [15]:
    __annotations__ = {'@context': {'@vocab': 'file://'}}

In [22]:
    def load_ipython_extension(shell):
        shell.display_formatter.mimebundle_formatter.for_type(requests.Response, annodize_response)   

In [23]:
    def unload_ipython_extension(shell):
        shell.display_formatter.mimebundle_formatter.for_type(requests.Response, lambda x: None)

In [ ]:
    shell = IPython.get_ipython()
    if shell:
        annotation = shell.user_ns[ANNO] = shell.user_ns.get(ANNO, {})
        if '@context' not in annotation:
            annotation.update({'@context': {'@vocab': 'file://'}})